In [14]:
import torch
import os
import pandas as pd
import csv
import glob
from torch import nn
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from models import *
from dataset import *

In [5]:
if torch.cuda.is_available():
    device='cuda:0'
    print('gpu found')
else:
    device='cpu'

gpu found


In [15]:
import gc
gc.collect()

143

In [21]:
N = 100

In [90]:

def train_epoch(train_loader, model, criterion, optimizer, val_loader):
    model.train()
    train_loss = 0.0
    for i, (X, y) in enumerate(train_loader):
        output = model(X)
        loss = criterion(output, y)
        train_loss += loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if i % 100 == 0:
            print(f"Training batch loss {train_loss}")
            
    val_loss = 0.0
    model.eval()     # Optional when not using Model Specific layer
    for data, labels in val_loader:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        target = model(data)
        loss = criterion(target,labels)
        val_loss += loss
        if i % 100 == 0:
            print(f"Training batch loss {val_loss}")
            

    

In [18]:
# TODO
gene = 'TP53'
patient_info_file = '../patient_id_all.txt'
with open(patient_info_file, 'r') as f:
    patient_list = f.read().splitlines()

label_info_file = '../genomic_data/label_summary.json'
with open(label_info_file, 'r') as f:
    label_dict = json.load(f)

feature_files = list(glob.glob('../features/*.npy'))
    
feature_dataset_train = tileFeatureDataset(feature_files, label_dict)

In [ ]:
f, l = feature_dataset[0]
f.shape

In [ ]:
# Start training
batch = 16
data_loader_train = DataLoader(feature_dataset_train, batch_size=batch, shuffle=True)
data_loader_val = DataLoader(feature_dataset_val, batch_size=batch, shuffle=True)
# hyperparameters
model = AttnClassifier.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()
epoch = 10
patients = 3

for e in range(epochs):
    train_loss = 0.0
    model.train()     # Optional when not using Model Specific layer
    for data, labels in data_loader_train:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        optimizer.zero_grad()
        target = model(data)
        loss = criterion(target,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    valid_loss = 0.0
    model.eval()     # Optional when not using Model Specific layer
    for data, labels in data_loader_val:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        
        target = model(data)
        loss = criterion(target,labels)
        valid_loss = loss.item() * data.size(0)

    print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(trainloader)} \t\t Validation Loss: {valid_loss / len(validloader)}')
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        # Saving State Dict
        torch.save(model.state_dict(), 'saved_model.pth')